In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import numpy as np
max_long = 25

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [5]:
!pip install unidecode

     |████████████████████████████████| 235 kB 12.4 MB/s 


In [6]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [7]:
import token_glove2
token_glove2.create_list, token_glove2.token_ize, token_glove2.voc_token, token_glove2.dict_token, token_glove2.token_tweet, token_glove2.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [8]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
df = data.get_data()
cleaned_df = data.clean_data(df)

In [11]:
text = token_glove2.create_list(cleaned_df['text'])
token = token_glove2.token_ize(text)
X = token_glove2.token_tweet(text, token)

In [12]:
word_vector_matrix = token_glove2.glove_vector(token)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [15]:
y = cleaned_df['target']

In [16]:
model = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model.add(Conv1D(128, 16, activation ='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=['accuracy'])
history = model.fit(X, y, batch_size=16, epochs = 22, validation_split=0.1, callbacks = [es])

Epoch 1/22
429/429 [==============================] - 15s 31ms/step - loss: 0.6814 - accuracy: 0.5751 - val_loss: 0.6029 - val_accuracy: 0.6798
Epoch 2/22
429/429 [==============================] - 14s 32ms/step - loss: 0.5659 - accuracy: 0.7269 - val_loss: 0.4991 - val_accuracy: 0.7979
Epoch 3/22
429/429 [==============================] - 13s 31ms/step - loss: 0.5053 - accuracy: 0.7701 - val_loss: 0.4687 - val_accuracy: 0.8045
Epoch 4/22
429/429 [==============================] - 13s 31ms/step - loss: 0.4784 - accuracy: 0.7916 - val_loss: 0.4456 - val_accuracy: 0.8097
Epoch 5/22
429/429 [==============================] - 13s 31ms/step - loss: 0.4504 - accuracy: 0.7994 - val_loss: 0.4387 - val_accuracy: 0.8045
Epoch 6/22
429/429 [==============================] - 13s 31ms/step - loss: 0.4322 - accuracy: 0.8140 - val_loss: 0.4356 - val_accuracy: 0.8045
Epoch 7/22
429/429 [==============================] - 13s 31ms/step - loss: 0.4099 - accuracy: 0.8260 - val_loss: 0.4343 - val_accuracy:

#Phase de Test

In [17]:
df = data.get_data(test=True)
test_df = data.clean_data(df)

In [18]:
text_test = token_glove2.create_list(test_df['text'])
token_test = token_glove2.token_ize(text_test)
X_test = token_glove2.token_tweet(text_test, token_test)

In [19]:
results = model.predict(X_test)

In [20]:
results

array([[0.3822587 ],
       [0.7833698 ],
       [0.8391292 ],
       ...,
       [0.24845475],
       [0.4237923 ],
       [0.27649465]], dtype=float32)

In [29]:
def arrondi(n):
  if n >= 0.5:
    return 1
  else :
    return 0

In [30]:
to_submit = [arrondi(x[0]) for x in results]

In [31]:
to_submit_df = pd.DataFrame(to_submit)

In [32]:
to_submit_df.rename(columns={0: 'target'}, inplace=True)

In [33]:
to_submit_df['id'] = test_df.id

In [34]:
to_submit_df = to_submit_df.set_index('id')

In [40]:
to_submit_df['target'].value_counts()

0    1981
1    1282
Name: target, dtype: int64

In [35]:
to_submit_df.to_csv('/content/gdrive/My Drive/submission6.csv')

In [28]:
model.save('/content/gdrive/My Drive/CNN_camille_1CSV_v1')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/CNN_camille_1CSV_v1/assets
